In [3]:
import jupyter_addons as ja
ja.set_css()

## Network Design with PypeFlow API - Example 2 
# Calculating Pressure Drops

In the first example of this series we used PypeFlow to determine the nominal diameters of a riser's pipe sections in an apartment building. We found that a nominal diameter (DN) of 40 mm would be appropriate. In this example we will add some fittings to the riser and take a look at the flow paths in the riser.  

## 1. Adding Fittings

### 1.1 Adding Tees

At each floor the riser has a branch off through which drinking water is distributed to the apartements on that floor. The branching is done with a tee whose straight leg has a nominal diameter of 40 mm and whose branch leg has a nominal diameter of 20 mm.<br>
<br>
The resistance coefficient of a tee can be calculated with the emperical formulas from CRANE, *Flow of Fluids Through Valves, Fittings and Pipe* (Technical Paper No. 410M) (2012). These formulas are implemented in PypeFlow. The module `pypeflow.core.resistance_coefficient` has a class called `Tee` that can be used to calculate the resistance coefficient of a particular tee. Take a look at the API documentation to learn how this class is used.

In [4]:
from pypeflow.core.resistance_coefficient import Tee
import quantities as qty
from pypeflow.core.pipe_schedules import PipeSchedule40

# input data

kwargs = {
    # the flow in the tee is diverging
    'flow_pattern': 'diverging',
    
    # diameters of the branch and combined leg
    'd_branch': PipeSchedule40.inside_diameter(qty.Length(20.0, 'mm')).get('mm'),
    'd_combined': PipeSchedule40.inside_diameter(qty.Length(40.0, 'mm')).get('mm'),
    
    # flow rates in the branch and combined leg
    'flow_rate_branch': qty.VolumeFlowRate(0.859, 'L/s').get('m^3/s'),
    'flow_rate_combined': qty.VolumeFlowRate(1.696, 'L/s').get('m^3/s'),
    
    # the branch leg is perpendicular to the straight leg
    'theta': qty.Angle(90.0, 'deg').get('deg')
}

# create Tee object

tee = Tee(**kwargs)


# get resistance coefficients of branch and straight leg

ja.display_item(f"Resistance coefficient of branch leg = <b>{tee.zeta_branch:.3f}</b>")
ja.display_item(f"Resistance coefficient of straight leg = <b>{tee.zeta_run:.3f}</b>")

These are the resistance coefficients for the first tee in the riser at node `n2` (see [scheme](../resources/ex2_scheme.pdf)). We should repeat the above also for the other tees in the riser:

In [7]:
import pandas as pd

nodes = [
    ('n3', 1.625, 0.859), 
    ('n4', 1.544, 0.859), 
    ('n5', 1.451, 0.859), 
    ('n6', 1.339, 0.859), 
    ('n7', 1.196, 0.859), 
    ('n8', 0.996, 0.889),
    ('n9', 0.556, 0.556)
]

tees = {'n2': tee}

for node in nodes:
    kwargs['flow_rate_branch'] = qty.VolumeFlowRate(node[2], 'L/s').get('m^3/s')
    kwargs['flow_rate_combined'] = qty.VolumeFlowRate(node[1], 'L/s').get('m^3/s')
    tees[node[0]] = Tee(**kwargs)

d = {
    'node': [node for node in tees.keys()],
    'zeta branch': [round(tees[node].zeta_branch, 3) for node in tees.keys()],
    'zeta run':[round(tees[node].zeta_run, 3) for node in tees.keys()]
}

df = pd.DataFrame(d)
ja.display_table(df)

,node,zeta branch,zeta run
0,n2,4.821,0.103
1,n3,4.737,0.112
2,n4,4.645,0.124
3,n5,4.545,0.140
4,n6,4.434,0.165
5,n7,4.304,0.206
6,n8,4.124,0.319
7,n9,4.057,0.400


### 1.2 Adding other fittings

In the first section of the riser, section `s12` between node `n1` and node `n2`, there are 5 elbows with a nominal diameter of 40 mm.
In CRANE Technical Paper No. 410M, Appendix A we find that an elbow has an Equivalent Length Ratio (ELR) equal to 30.<br>
<br>
There are also a swing check valve and two ball valves in this section, whose ELR's are:
- swing check valve: ELR = 100
- ball valve (full port): ELR = 3

### 1.3 Creating a fitting file

Once we have gathered all information about the fittings that are present in the pipe sections, we can create a fitting csv-file by which we will pass this information to PypeFlow's `Designer`. Make sure that when saving the file the decimal separator is a point, not a comma. Let's have a look at the fitting file in this example:

In [8]:
df = pd.read_csv('../projects/config3_fittings.csv')
ja.display_table(df)

,section_id,fitting_id,type,zeta,zeta_inf,zeta_d,ELR,Kv
0,s12,cv1,check_valve,NaN,NaN,NaN,100.0,NaN
1,s12,bv1,ball_valve,NaN,NaN,NaN,3.0,NaN
2,s12,bv2,ball_valve,NaN,NaN,NaN,3.0,NaN
3,s12,elb1,elbow,NaN,NaN,NaN,30.0,NaN
4,s12,elb2,elbow,NaN,NaN,NaN,30.0,NaN
5,s12,elb3,elbow,NaN,NaN,NaN,30.0,NaN
6,s12,elb4,elbow,NaN,NaN,NaN,30.0,NaN
7,s12,elb5,elbow,NaN,NaN,NaN,30.0,NaN
8,s12,tee_n2,tee,0.103,NaN,NaN,NaN,NaN
9,s23,tee_n3,tee,0.112,NaN,NaN,NaN,NaN


Notice that Pandas indicates empty cells in the table as `NaN` (Not a Number).<br>
<br>
In pipe section `s12` (between nodes `n1` and `n2`) we have added the check valve, two ball valves and five elbow fittings. For these valves and fittings we found the ELR in CRANE Technical Paper No. 410M, Appendix A. The resistance coefficient `zeta` of the straight leg of the tee in node `n2` was calculated above. In the following pipe sections we have added the resistance coefficients of the straight legs of the other tees that were also calculated above.

## 2. Preparing the Network Configuration File

We know the design peak flow rates in the pipe sections of the riser and we know their nominal diameter. We will use the same nominal diameter of 40 mm for all pipe sections of the riser, as we have discussed in our first design example. The csv network configuration file now looks like this:

In [9]:
df = pd.read_csv('../projects/config2_pressure.csv')
ja.display_table(df)

,section_id,start_node_id,start_node_height,end_node_id,end_node_height,length,diameter_nom,flow_rate,pressure_drop
0,s12,n1,0.0,n2,4.4,16.7,40.0,1.696,NaN
1,s23,n2,4.4,n3,7.1,2.7,40.0,1.625,NaN
2,s34,n3,7.1,n4,9.8,2.7,40.0,1.544,NaN
3,s45,n4,9.8,n5,12.5,2.7,40.0,1.451,NaN
4,s56,n5,12.5,n6,15.2,2.7,40.0,1.339,NaN
5,s67,n6,15.2,n7,17.9,2.7,40.0,1.196,NaN
6,s78,n7,17.9,n8,20.6,2.7,40.0,0.996,NaN
7,s89,n8,20.6,n9,23.3,2.7,40.0,0.556,NaN
8,s90,n9,23.3,n0,0.0,NaN,NaN,NaN,NaN
9,s80,n8,20.6,n0,0.0,NaN,NaN,NaN,NaN


We reworked our previous configuration file from example 1 (and saved it under a new name). In column `diameter_nom` we have entered the nominal diameter of the pipe sections. The column `pressure_drop` is now empty.

## 3. Putting the `Designer` to work

The same procedure as in example 1 applies, to which we now also must add the fitting file:
1. Set the measuring units.
2. Let the `Designer` create the `Network` object.
3. Pass the network configuration file to the `Designer`.
4. Pass the fitting file to the `Designer`.

In [10]:
from pypeflow.design import Designer

Designer.set_units({
    'length': 'm',
    'diameter': 'mm',
    'flow_rate': 'L/s',
    'pressure': 'bar',
    'velocity': 'm/s'
})

Designer.create_network(
    start_node_id='n1',
    end_node_id='n0',
    fluid='water',
    fluid_temperature=10.0,
    pipe_schedule='pipe_schedule_40'
)

Designer.configure_network('../projects/config2_pressure.csv')

Designer.add_fittings('../projects/config3_fittings.csv')

## 4. Looking at the Results

### 4.1 Checking the Pressure Losses in the Pipe Sections

In [11]:
df_sections = Designer.get_sections()
ja.display_table(df_sections)

,section_id,L [m],"Di,th [mm]",Di [mm],DN [mm],V [L/s],v [m/s],"dp,dyn [bar]"
0,s12,16.7,40.9,40.9,40.0,1.696,1.291,0.128
1,s23,2.7,40.9,40.9,40.0,1.625,1.237,0.013
2,s34,2.7,40.9,40.9,40.0,1.544,1.175,0.012
3,s45,2.7,40.9,40.9,40.0,1.451,1.104,0.011
4,s56,2.7,40.9,40.9,40.0,1.339,1.019,0.010
5,s67,2.7,40.9,40.9,40.0,1.196,0.910,0.008
6,s78,2.7,40.9,40.9,40.0,0.996,0.758,0.006
7,s89,2.7,40.9,40.9,40.0,0.556,0.423,0.002
8,s90,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,s80,NaN,NaN,NaN,NaN,NaN,NaN,NaN


From the table above we can read off the flow velocity in each pipe section of the riser and the dynamic pressure loss caused by the flow of water during peak flow rate conditions in the pipe section.

### 4.2 Having a Look at the Fittings in the Network

In [12]:
df_fittings = Designer.get_fittings()
ja.display_table(df_fittings)

,section_id,fitting_id,dp [bar],zeta,zeta_inf,zeta_d,ELR,Kv
0,s12,cv1,0.017,NaN,NaN,NaN,100.0,NaN
1,s12,bv1,0.001,NaN,NaN,NaN,3.0,NaN
2,s12,bv2,0.001,NaN,NaN,NaN,3.0,NaN
3,s12,elb1,0.005,NaN,NaN,NaN,30.0,NaN
4,s12,elb2,0.005,NaN,NaN,NaN,30.0,NaN
5,s12,elb3,0.005,NaN,NaN,NaN,30.0,NaN
6,s12,elb4,0.005,NaN,NaN,NaN,30.0,NaN
7,s12,elb5,0.005,NaN,NaN,NaN,30.0,NaN
8,s12,tee_n2,0.001,0.103,NaN,NaN,NaN,NaN
9,s23,tee_n3,0.001,0.112,NaN,NaN,NaN,NaN


In this table we can see what pressure loss is caused by each fitting.

### 4.3 Checking the Flow Paths in the Network

In [13]:
df_paths = Designer.get_paths()
ja.display_table(df_paths)

,path,"dp,vel [bar]","dp,elev [bar]","dp,dyn [bar]","dp,stat req. [bar]","dp,dif [bar]"
1,s12|s20,0.000,0.432,0.128,0.560,1.909
2,s12|s23|s30,-0.001,0.696,0.142,0.837,1.631
3,s12|s23|s34|s40,-0.001,0.961,0.154,1.114,1.355
4,s12|s23|s34|s45|s50,-0.002,1.226,0.165,1.389,1.080
5,s12|s23|s34|s45|s56|s60,-0.003,1.491,0.175,1.662,0.806
6,s12|s23|s34|s45|s56|s67|s70,-0.004,1.755,0.183,1.934,0.535
7,s12|s23|s34|s45|s56|s67|s78|s80,-0.005,2.020,0.189,2.203,0.265
0,s12|s23|s34|s45|s56|s67|s78|s89|s90,-0.007,2.285,0.191,2.468,0.000


To understand the results in the table above, we can refer to the energy equation applied along the path. The energy equation for an incompressible fluid in case of pipe flow can be written down as:

$$
\left( {{p_1} + \rho \frac{{v_1^2}}{2} + \rho g{z_1}} \right) + \Delta {p_P} - \Delta {p_L} = \left( {{p_2} + \rho \frac{{v_2^2}}{2} + \rho g{z_2}} \right)
$$

where:

- $p_1$ = static pressure at the entrance of the flow path
- $\rho \frac{{v_1^2}}{2}$ = velocity pressure at the entrance of the flow path
- $\rho g{z_1}$ = elevation pressure at the entrance of the flow path
- $\Delta{p_P}$ = amount of pressure added by pumps along the flow path
- $\Delta {p_{L}}$ = all pressure losses due to flow along the flow path
- ${p_2}$ = static pressure at the exit of the flow path
- $\rho \frac{{v_2^2}}{2}$ = velocity pressure at the exit of the flow path
- $\rho g{z_2}$ = elevation pressure at the exit of the flow path

The terms between brackets are the total pressure of the fluid (i.e. mechanical energy of the fluid in terms of pressure) at the entrance of the flow path, respectively at the exit of the flow path. The energy equation states that the total pressure of the fluid at the exit of the flow path follows from the total pressure of the fluid at the entrance of the flow path from which the pressure losses (i.e. the mechanical energy losses in terms of pressure) due to flow along the flow path are subtracted and to which the amount of pressure supplied by any pumps present in the flow path (i.e. the mechanical energy input in terms of pressure) is added.

The energy equation can also be written like this:

$$
{p_1} - {p_2} = \frac{\rho }{2}\left( {v_2^2 - v_1^2} \right) + \rho g\left( {{z_2} - {z_1}} \right) + \Delta {p_L} - \Delta {p_P}
$$
$$
\Delta {p_s} = \Delta {p_v} + \Delta {p_z} + \Delta {p_L} - \Delta {p_P}
$$

where:

- $\Delta {p_s} = {p_1} - {p_2}$ = static head between entrance and exit of the flow path
- $\Delta {p_v} = \frac{\rho }{2}\left( {v_2^2 - v_1^2} \right)$ = velocity head between exit and entrance of the flow path
- $\Delta {p_z} = \rho g\left( {{z_2} - {z_1}} \right)$ = elevation head between exit and entrance of the flow path


In the table above,
- Column `dp,vel` refers to the velocity head. It can be noticed that it is small with respect to the other terms. Therefore, it is generally often neglected.
- Column `dp,elev` refers to the elevation head.
- Column `dp,dyn` refers to the dynamic head, which is equal to $\Delta {p_L} - \Delta {p_P}$ and thus also includes the pump term. As no pump has been considered $\Delta {p_P}$ = 0 applies.
- Column `dp,stat req.` refers to the required static head between the entrance and exit of the flow path in order to overcome the other heads that are on the right side of the equation.
- Column `dp,dif` is the difference between the static head of the critical path, i.e. the path with the greatest required static head, and the path under consideration.

As it could be expected, the longest and highest flow path requires the greatest static head, especially because of its height. Based on this result, it can be concluded that in order to maintain a feed pressure of 3 bar at the highest point of the riser (see design example 1), we would need a feed pressure of at least 5,5 bar at the bottom of the riser.